In [1]:
import sqlite3
import requests
import datetime
import pandas as pd 

In [8]:
ACCESS_TOKEN = os.environ.get('ACCESS_TOKEN')

In [1]:
teams = {"Dodgers" : {"teamID": 1061, "venueID": 744},
         "Angels": {"teamID": 1062, "venueID" : 9763},
         "Giants" : {"teamID" : 197, "venueID" : 82}}

In [4]:
for team in teams:
    print(teams[team]["teamID"])

1061
1062
197


In [5]:
def get_ticket_info(teamID, venueID):
    url = f"https://api.stubhub.com/sellers/search/events/v3?venueId={venueID}&performerId={teamID}&parking=false&sort=eventDateLocal%20asc&rows=81"
    headers = { 
    "Authorization": "Bearer " + ACCESS_TOKEN,
    "Accept": "application/json"
    }
    r = requests.get(url, headers=headers)
    return r

def create_df(r):
    events = r.json()['events']
    prices = {event['id']:{
         "price": event['ticketInfo']['minPrice'],
         "time" : event['eventDateLocal'],
         "name" : event['name'],
         "extractTime":datetime.datetime.today(),
         "homeTeam" : event['performers'][0]['name'],
         "awayTeam" : event['performers'][1]['name']} for event in events}
    df = pd.DataFrame(prices).T.reset_index().rename({"index":"id"}, axis = 1)
    df['time'] = pd.to_datetime(df['time'].str[0:10])
    df['days_to_game'] = (df['time'] - df['extractTime'])
    df['days_to_game'] = df['days_to_game'].apply(lambda X: X.days)
    return df

In [11]:
for team in teams:
    teamID = teams[team]["teamID"]
    venueID = teams[team]["venueID"]
    r = get_ticket_info(teamID, venueID)
    df = create_df(r)
    with sqlite3.connect("../tickets.db") as conn: 
        df.to_sql("extraction", conn, if_exists = "append", index = False)

In [2]:
with sqlite3.connect("tickets.db") as conn: 
    cursor = conn.cursor()
    cursor.execute("SELECT sql FROM sqlite_master WHERE type='table';")

In [61]:
for result in cursor.fetchall():
    print(result[0])

CREATE TABLE "extraction" (
"price" REAL,
  "time" TEXT,
  "name" TEXT,
  "extractTime" TIMESTAMP
)


In [3]:
with sqlite3.connect("../tickets.db") as conn: 
    cmd = "SELECT * FROM extraction"
    df2 = pd.read_sql_query(cmd, conn)

In [4]:
df2

,id,price,time,name,extractTime,homeTeam,awayTeam,days_to_game
0,104722879,58.62,2021-06-11 00:00:00,Texas Rangers at Los Angeles Dodgers,2021-06-08 17:51:48.284897,Los Angeles Dodgers,Texas Rangers,2
1,104725408,29.16,2021-06-12 00:00:00,Texas Rangers at Los Angeles Dodgers,2021-06-08 17:51:48.284897,Los Angeles Dodgers,Texas Rangers,3
2,104725409,28.04,2021-06-13 00:00:00,Texas Rangers at Los Angeles Dodgers,2021-06-08 17:51:48.284897,Los Angeles Dodgers,Texas Rangers,4
3,104724531,35.55,2021-06-14 00:00:00,Philadelphia Phillies at Los Angeles Dodgers,2021-06-08 17:51:48.284897,Los Angeles Dodgers,Philadelphia Phillies,5
4,104725410,24.99,2021-06-15 00:00:00,Philadelphia Phillies at Los Angeles Dodgers,2021-06-08 17:51:48.284897,Los Angeles Dodgers,Philadelphia Phillies,6
...,...,...,...,...,...,...,...,...
150,104738380,17.17,2021-09-29 00:00:00,Arizona Diamondbacks at San Francisco Giants,2021-06-08 17:51:49.782776,San Francisco Giants,Arizona Diamondbacks,112
151,104736599,17.17,2021-09-30 00:00:00,Arizona Diamondbacks at San Francisco Giants,2021-06-08 17:51:49.782776,San Francisco Giants,Arizona Diamondbacks,113
152,104736601,51.46,2021-10-01 00:00:00,San Diego Padres at San Francisco Giants,2021-06-08 17:51:49.782776,San Francisco Giants,San Diego Padres,114
153,104738382,65.83,2021-10-02 00:00:00,San Diego Padres at San Francisco Giants,2021-06-08 17:51:49.782776,San Francisco Giants,San Diego Padres,115


In [6]:
df2.pivot_table(values = "price", columns = "days_to_game", index = "name")

days_to_game,2,3,4,5,6,7,15,16,17,18,...,95,96,97,98,111,112,113,114,115,116
name,,,,,,,,,,,,,,,,,,,,,
Arizona Diamondbacks at Los Angeles Dodgers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,36.54,21.64,27.86,NaN,NaN,NaN,NaN,NaN,NaN
Atlanta Braves at Los Angeles Dodgers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chicago Cubs at Los Angeles Dodgers,NaN,NaN,NaN,NaN,NaN,NaN,46.19,44.96,48.92,36.37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Colorado Rockies at Los Angeles Dodgers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Houston Astros at Los Angeles Dodgers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Los Angeles Angels at Los Angeles Dodgers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Milwaukee Brewers at Los Angeles Dodgers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.14,44.96,32.69
New York Mets at Los Angeles Dodgers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Philadelphia Phillies at Los Angeles Dodgers,NaN,NaN,NaN,35.55,25.32,39.29,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
df['price'] = df['price'].astype("float32")

In [116]:
conn.close()

In [7]:
df2

,id,price,time,name,extractTime,days_to_game
0,104722879,58.62,2021-06-11 00:00:00,Texas Rangers at Los Angeles Dodgers,2021-06-08 21:51:33.574579,2
1,104725408,31.46,2021-06-12 00:00:00,Texas Rangers at Los Angeles Dodgers,2021-06-08 21:51:33.574589,3
2,104725409,28.04,2021-06-13 00:00:00,Texas Rangers at Los Angeles Dodgers,2021-06-08 21:51:33.574593,4
3,104724531,35.55,2021-06-14 00:00:00,Philadelphia Phillies at Los Angeles Dodgers,2021-06-08 21:51:33.574595,5
4,104725410,25.32,2021-06-15 00:00:00,Philadelphia Phillies at Los Angeles Dodgers,2021-06-08 21:51:33.574597,6
...,...,...,...,...,...,...
99,104724625,36.37,2021-09-29 00:00:00,San Diego Padres at Los Angeles Dodgers,2021-06-08 22:15:26.493996,112
100,104724627,35.14,2021-09-30 00:00:00,San Diego Padres at Los Angeles Dodgers,2021-06-08 22:15:26.493997,113
101,104724629,35.14,2021-10-01 00:00:00,Milwaukee Brewers at Los Angeles Dodgers,2021-06-08 22:15:26.493999,114
102,104724631,44.96,2021-10-02 00:00:00,Milwaukee Brewers at Los Angeles Dodgers,2021-06-08 22:15:26.494000,115
